In [ ]:
# install pandas dependencies
%pip install pandas
%pip install matplotlib


In [ ]:
# import pandas
import pandas as pd
import numpy
#matplotlib
import matplotlib.pyplot as plt
import re


In [ ]:
# download zip urls into ../data/ folder
!rm -rf ./work/data/*
!wget -P ./work/data "https://github.com/JaimeObregon/subvenciones/raw/main/files/convocatorias.csv.gz"
!wget -P ./work/data "https://github.com/JaimeObregon/subvenciones/raw/main/files/juridicas_1.csv.gz"
!wget -P ./work/data "https://github.com/JaimeObregon/subvenciones/raw/main/files/juridicas_2.csv.gz"


In [ ]:
# unzip all files
!gunzip ./work/data/*.gz -d


In [ ]:
!ls -lha ./work/data/

In [ ]:
# setup pandas plot background to a gray one
plt.style.use('dark_background')

# show 100000000 instead of 1e8
pd.options.display.float_format = '{:,.0f}'.format

In [ ]:
# Files:
# convocatorias.csv
# juridicas_1.csv
# juridicas_2.csv
# load data into pandas dataframes
convocatorias_columns = ["id",
    "codigo_bdns",
    "MRR",
    "administracion",
    "departamento",
    "organo",
    "fecha_registro",
    "titulo_convocatoria",
    "url_bases_regul",
    "titulo_cooficial",
    "colDesconocida1",
    "colDesconocida2",
    "colDesconocida3"]

convocatorias = pd.read_csv('./data/convocatorias.csv', names=convocatorias_columns, header=0)
# juridicas_2 = pd.read_csv('./data/juridicas_2.csv')convocatorias['departamento']
# drop columns colDesconocida1, colDesconocida2, colDesconocida3
convocatorias = convocatorias.drop(columns=['colDesconocida1', 'colDesconocida2', 'colDesconocida3'])
convocatorias['MRR'] = convocatorias['MRR'] == 'SI'
convocatorias['fecha_registro'] = pd.to_datetime(convocatorias['fecha_registro'], format='%d/%m/%Y')
convocatorias['year'] = convocatorias['fecha_registro'].dt.year
convocatorias.info()



In [ ]:
juridicas_columns = ["id",
    "id_convocatoria",
    "administracion",
    "departamento",
    "organo",
    "titulo_convocatoria",
    "url_bases_regul",
    "apli_presupuestaria",
    "fecha_concesion",
    "beneficiario",
    "importe",
    "instrumento",
    "ayuda_equivalente",
    "detalles",
    "codigo_bdns",
    "valor_desconocido",
    "id_interna"]
juridicas = pd.read_csv('./data/juridicas_1.csv', names=juridicas_columns, header=0)
# la columna beneficiario contiene el id (nif) del beneficiario en los primeros 9 caracteres
juridicas['beneficiario_id'] = juridicas['beneficiario'].str.extract(r'([A-Z0-9]{9})')
juridicas['beneficiario'] = juridicas['beneficiario'].str.replace(r'(^[A-Z0-9]{9})', '')
juridicas['fecha_concesion'] = pd.to_datetime(juridicas['fecha_concesion'], format='%d/%m/%Y')
juridicas['year'] = juridicas['fecha_concesion'].dt.year
juridicas = juridicas.reindex(sorted(juridicas.columns), axis=1)
juridicas.info()

In [ ]:
# 
juridicas['beneficiario'].value_counts().sort_values(ascending=False).head(10)

In [ ]:
# juridicas.groupby('instrumento')['importe'].sum().sort_values(ascending=False).head(10)
# also group by beneficiario
grouped_jur =  juridicas.groupby(['beneficiario', 'instrumento'])['importe'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)
grouped_jur.head(10)


In [ ]:
# este es el top 10 de beneficiarios que más dinero han recibido y en 1 pago!
# juridicas.query('beneficiario.str.contains("Mº SANIDAD CONSUMO Y BIENESTAR SOCIAL")')
juridicas.query('beneficiario.str.contains("EL CORTE INGLES, S.A.")')

In [69]:
top_ten_beneficiarios = juridicas.groupby('beneficiario')['importe'].sum().sort_values(ascending=False).head(10)

In [ ]:
grouped_jur.sort_values(by='count', ascending=False).head(10).plot.barh(y='count')

In [ ]:
# merge convocatorias en juridicas by id_convocatoria
convocatorias_juridicas = pd.merge(convocatorias, juridicas, left_on='id', right_on='id_convocatoria')

In [72]:
# cantidad de convocatorias por mes
convocatorias['fecha_registro'].dt.month.value_counts().sort_index()\
    .plot(kind='bar', title='Convocatorias por mes', figsize=(10, 5)).set_xticklabels(['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre'], rotation=45)

plt.show()

NameError: name 'convocatorias' is not defined

In [ ]:
# Convocatorias por administracion
convocatorias['administracion'].value_counts().head(15).sort_values().plot(kind='barh')
plt.show()


In [ ]:
convocatorias['departamento']